In [1]:
# !pip install -U blingfire
import blingfire

     |████████████████████████████████| 2.8MB 59kB/s eta 0:00:013


In [3]:
text = 'This is the Bling-Fire tokenizer'
output = text_to_ids(text)
print(output)

TypeError: text_to_ids() missing 2 required positional arguments: 's' and 'max_len'

In [1]:
import csv
import sys
from pytorch_transformers import BertTokenizer
from multiprocessing import Pool
from tqdm import tqdm_notebook as tqdm
import json
import os

UNK = "[UNK]"
BOS = "[CLS]"
EOS = "[SEP]"
PAD = "[PAD]"

_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True, never_split = ["<br/>"])
def subtask(s):
    ts = [BOS] + _tokenizer.tokenize(s) + [EOS]
    idxs = _tokenizer.convert_tokens_to_ids(ts)
    #assert all((i >= 0 for i in idxs))
    return idxs

class Processor:
    def __init__(self, threads):
        self.score = []
        self.summary = []
        self.text = []
        self.n_worker = threads        
        self.size = 0
    
    def convert_all_to_ids(self):
        if self.n_worker > 1:
            with Pool(self.n_worker) as p:
                chunksize = 100
                sumseqs = list(tqdm(p.imap(subtask, self.summary, chunksize=chunksize), total=self.size))
        else:
            sumseqs = [subtask(s) for s in tqdm(self.summary)]
        if self.n_worker > 1:
            with Pool(self.n_worker) as p:
                chunksize = 100
                tseqs = list(tqdm(p.imap(subtask, self.text, chunksize=chunksize), total=self.size))
        else:
            tseqs = [subtask(s) for s in tqdm(self.text)]
        self.textseqs = tseqs
        self.sumseqs = sumseqs
        
    def __len__(self):
        return self.size
    
    def dump(self, name, vocabname):
        seqdata = {"text":self.textseqs, "summary":self.sumseqs, "score":self.score}
        json.dump(seqdata, open(name, 'w'))
        if vocabname is not None:
            json.dump(_tokenizer.vocab, open(vocabname, 'w'))
    
    @staticmethod
    def readcsv(input_file, quotechar="\""):
        """Reads a comma separated value file."""
        with open(input_file, "r", encoding="utf-8") as f:
            reader = csv.reader(f, delimiter=",", quotechar=quotechar)
            lines = []
            for line in reader:
                if sys.version_info[0] == 2:
                    line = list(unicode(cell, 'utf-8') for cell in line)
                lines.append(line)
            return lines
        
class FoodProcessor(Processor):
     def __init__(self, name, threads):
        super(FoodProcessor, self).__init__(threads)
        lines = self.readcsv(name)

        for d in lines[1:]:
            #self.score.append(int(d[6]) > 3) # convert to 0/1
            self.score.append(int(d[6])-1)
            self.summary.append(d[8])
            self.text.append(d[9].strip("\""))
        self.size = len(self.text)   
            
class IMDBProcessor(Processor):
    def __init__(self, dirname, threads):
        super(IMDBProcessor, self).__init__(threads)
        
        posdir = os.path.join(dirname, 'pos')
        negdir = os.path.join(dirname, 'neg')
        
        self.score = []
        self.text = []
        self.summary = []
        
        for lb,subdir in enumerate(['neg','pos']):
            subdir = os.path.join(dirname, subdir)
            for entry in os.listdir(subdir):
                fname = os.path.join(subdir, entry)
                with open(fname, 'r') as f:
                    t = f.read().strip()
                self.score.append(lb)
                self.text.append(t)
                self.summary.append(t)
        
        self.size = len(self.text)
            
class BBCProcessor(Processor):
     def __init__(self, name, threads):
        super().__init__(threads)
        lines = self.readcsv(name)
        labeldict = {}
        for lb, txt in lines[1:]:
            labeldict[lb] = labeldict.get(lb, len(labeldict))
            self.score.append(labeldict[lb]) # convert to 0/1
            self.summary.append(txt)
            self.text.append(txt)
        self.size = len(self.text)   

In [2]:
p = FoodProcessor("data/Reviews.csv", threads=12)#
# p = IMDBProcessor("data/aclImdb/train/", threads=12)
# p = BBCProcessor("data/BBC/bbc-text.csv", threads=4)

In [ ]:
p.convert_all_to_ids()

Token indices sequence length is longer than the specified maximum sequence length for this model (531 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (627 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (832 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (626 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1425 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for th

Token indices sequence length is longer than the specified maximum sequence length for this model (631 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (528 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (660 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (572 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (587 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for thi

Token indices sequence length is longer than the specified maximum sequence length for this model (844 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (674 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (2534 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (634 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (548 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for th

Token indices sequence length is longer than the specified maximum sequence length for this model (625 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (588 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (792 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (728 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (766 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for thi

Token indices sequence length is longer than the specified maximum sequence length for this model (1180 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (919 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (664 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (677 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1012 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for t

...

Token indices sequence length is longer than the specified maximum sequence length for this model (570 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (622 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (700 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (583 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (522 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for thi

Token indices sequence length is longer than the specified maximum sequence length for this model (567 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (798 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (539 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (807 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (567 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for thi

Token indices sequence length is longer than the specified maximum sequence length for this model (619 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (579 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (538 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (725 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (524 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for thi

Token indices sequence length is longer than the specified maximum sequence length for this model (818 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1591 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (752 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1009 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (752 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for t

Token indices sequence length is longer than the specified maximum sequence length for this model (570 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1033 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1811 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (706 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (513 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for t

In [ ]:
from matplotlib import pyplot as plt
lplot = [len(s) for s in p.textseqs]
plt.hist(lplot, bins=50)
plt.show()

In [ ]:
import numpy as np
mean = np.mean(lplot)
std = np.std(lplot)
print(mean,std) 

In [ ]:
#p.dump("data/BBC/data.json", "data/BBC/vocab.json")
#p.dump("data/IMDB/data.json", "data/IMDB/vocab.json")
p.dump("data/food/data.json", "data/food/vocab.json")

In [ ]:
from matplotlib import pyplot as plt
lplot = p.score
plt.hist(lplot, bins=50)
plt.show()